Importamos las librerias necesarias

In [1]:
import pandas as pd
import os

Obtenemos la ruta de la carpeta en la que estamos trabajando

Nos servira para abrir los archivos y luego guardarlos

In [2]:
# Construir la ruta completa al archivo
ruta_items = r'C:\PROYECTO-INDIVIDUAL\Data\json\sucio\australian_users_items.json'

# Abrimos el archivo y cargamos todas las lineas del mismo en "lineas"
with open(ruta_items, 'r', encoding='utf-8') as archivo:
    lineas = archivo.readlines()

# Leemos cada linea dentro de lineas, y realizamos la correcciones pertinentes
texto = []
for linea in lineas:
    texto_de_linea = linea.replace("'", '"').replace('"', '').replace('user_id: ',  '"user_id": "').replace(', items_count: ',  '", "items_count": "').replace(', steam_id:',  '", "steam_id": "').replace(', user_url:',  '", "user_url": "').replace(', items:',  '", "items": ').replace('item_id:',  '"item_id": "').replace(', item_name:',  '", "item_name": "').replace(', playtime_forever:',  '", "playtime_forever": "').replace(', playtime_2weeks:',  '", "playtime_2weeks": "').replace('},', '"},').replace('}]', '"}]').replace("\\", "")
    texto.append(texto_de_linea)

# Abrimos el archivo en formato escritura, cargamos todo el texto, y guardamos en una nueva ruta
ruta_nueva_items = r'C:\PROYECTO-INDIVIDUAL\Data\json\limpio\items.json'
archivo = open(ruta_nueva_items, "w", encoding='utf-8')
archivo.write('[' + '\n')
for i in range(len(texto)-1):
    archivo.write(texto[i] + ',')
archivo.write(texto[len(texto)-1])
archivo.write(']' + '\n')
archivo.close()

CREAMOS UN DATAFRAME CON ESE ARCHIVO

In [3]:
df_items = pd.read_json(ruta_nueva_items, encoding='utf-8')

REALIZAMOS UNA VISUALIZACION DE LOS DATOS, FILAS Y COLUMNAS

Con el fin de saber que columnas nos pueden servir de acuerdo a las actividades a realizar

In [4]:
print(df_items.shape)
print(df_items.columns)

(88310, 5)
Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')


AHORA ELIMINAREMOS COLUMNAS, VALORES NA, Y MODIFICAREMOS DATOS

In [5]:
# Dropeamos la columna 'user_url'
df_items.drop(inplace=True, columns=['user_url'])

# Separamos la columna items
df_items = df_items.explode(column='items')

# Creamos listas donde se guardara todos los datos de lo que sera el nuevo df
lista_item_id = []
lista_item_name = []
lista_playtime_forever = []
lista_playtime_2weeks = []
for x in range(len(df_items['items'])):
    item = df_items['items'].iloc[x]
    if pd.notna(item):
        diccionario = dict(item)
        if 'item_name' in diccionario:
            lista_item_name.append(diccionario['item_name'])
        else:
            lista_item_name.append('None')
        if 'item_id' in diccionario:
            lista_item_id.append(diccionario['item_id'])
        else:
            lista_item_id.append('None')
        if 'playtime_forever' in diccionario:
            lista_playtime_forever.append(diccionario['playtime_forever'])
        else:
            lista_playtime_forever.append('None')
        if 'playtime_2weeks' in diccionario:
            lista_playtime_2weeks.append(diccionario['playtime_2weeks'])
        else:
            lista_playtime_2weeks.append('None')
    else:
        lista_item_name.append('None')
        lista_item_id.append('None')
        lista_playtime_forever.append('None')
        lista_playtime_2weeks.append('None')

# Creamos las nuevas columnas con las listas hechas anteriormente
df_items['item_id'] = lista_item_id
df_items['item_name'] = lista_item_name
df_items['playtime_forever'] = lista_playtime_forever
df_items['playtime_2weeks'] = lista_playtime_2weeks

df_items['user_id'] = df_items['user_id'].str.strip()
df_items['item_id'] = df_items['item_id'].str.strip()

# Dropeamos la columna items ya que no la necesitamos
df_items.drop(columns=['items', 'playtime_2weeks', 'item_name', 'items_count', 'steam_id'], inplace=True)

# Transformamos los valores 'None' por None en la columna item_id
df_items.loc[df_items['item_id'].astype(str).str.contains('None'), 'item_id'] = None
# Eliminamos los valores NA
df_items.dropna(subset=['item_id'], inplace=True)
# Transformamos a int la columna
df_items['item_id'] = df_items['item_id'].astype(int)

df_items['playtime_forever'] = df_items['playtime_forever'].str.strip()
df_items.loc[df_items['playtime_forever'].astype(str).str.contains('None'), 'playtime_forever'] = None
df_items.dropna(subset=['playtime_forever'], inplace=True)
df_items['playtime_forever'] = df_items['playtime_forever'].astype(int)
df_items = df_items[df_items['playtime_forever'] != 0]

Vemos como quedo

In [6]:
df_items.head()

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
0,76561197970982479,30,7
0,76561197970982479,300,4733
0,76561197970982479,240,1853
0,76561197970982479,3830,333


In [7]:
df_items

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
0,76561197970982479,30,7
0,76561197970982479,300,4733
0,76561197970982479,240,1853
0,76561197970982479,3830,333
...,...,...,...
88308,76561198329548331,304930,677
88308,76561198329548331,227940,43
88308,76561198329548331,388490,3
88308,76561198329548331,521570,4


GUARDAMOS LOS DATAFRAMES

Para ello creamos una carpeta llamada parquet dentro de datos, y guardamos los archivos dentro de ella

In [8]:
ruta_de_guardado = r'C:\PROYECTO-INDIVIDUAL\Data\parquet\items.parquet'
df_items.to_parquet(ruta_de_guardado, index=False)